# Цветовые схемы. Преобразования цветовых схем.

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np

import random
import pathlib

from matplotlib.colors import LinearSegmentedColormap
import scipy.spatial.transform.rotation

In [ ]:
path = "img/medium"
collection = images = [p for p in pathlib.Path(path).iterdir() if p.suffix in [".jpg", ".jpeg"]]
img = plt.imread(random.choice(collection))

plt.close(fig=1)
plt.figure(num=1)
plt.gca().imshow(img)
plt.gca().set_axis_off()
plt.show()

In [ ]:
colormaps = [
    ["Reds", "Greens", "Blues"],
    [
        LinearSegmentedColormap.from_list("BkR", ["Black", "Red"]),
        LinearSegmentedColormap.from_list("BkG", ["Black", "Green"]),
        LinearSegmentedColormap.from_list("BkBu", ["Black", "Blue"])
    ],
]
plt.close(fig=2)
plt.figure(num=2)
for cm in range(len(colormaps)):
    for ch in range(len(colormaps[cm])):
        plt.subplot(2, 3, cm*3 + ch+1).imshow(img[:, :, ch], cmap=colormaps[cm][ch])

for axes in plt.gcf().findobj(plt.Axes):
    axes.set_axis_off()
plt.show()

In [ ]:
# RGB cube

granularity = 8
edge_len= int(256 / granularity)
color_sl = complex(0, edge_len + 1)
cube_grid = np.mgrid[0:1:color_sl, 0:1:color_sl, 0:1:color_sl]
cube = (cube_grid[0] < 1) & (cube_grid[1] < 1) & (cube_grid[2] < 1)

# values are the same (laying in range [0, 1])
colors = np.transpose(cube_grid, (1, 2, 3, 0))

plt.close(fig=3)
plt.figure(num=3).add_subplot(projection='3d')
plt.gca().voxels(cube_grid[0], cube_grid[1], cube_grid[2], # grid points positions along x, y and z axes
                 cube[:-1, :-1, :-1], facecolors=colors[:-1, :-1, :-1], # where to fill grid and corresponing colors
                 shade=False)

plt.gca().set_xlabel("R")
plt.gca().set_ylabel("G")
plt.gca().set_zlabel("B")
plt.show()

In [ ]:
#CMY cube

plt.close(fig=4)
plt.figure(num=4).add_subplot(projection='3d')
plt.gca().voxels(cube_grid[0], cube_grid[1], cube_grid[2], 
                 cube[:-1, :-1, :-1], facecolors=(1 - colors[:-1, :-1, :-1]), 
                 shade=False)

plt.gca().set_xlabel("C")
plt.gca().set_ylabel("M")
plt.gca().set_zlabel("Y")
plt.show()

In [ ]:
z_axis_rot = -np.pi/6 # introduce rotation in xOy plane to avoid equations matrix singularity
cube_rotation = scipy.spatial.transform.Rotation.from_euler("xyz", (np.pi / 4, -np.arctan(1/np.sqrt(2)), z_axis_rot))
cube_scaled_grid = np.transpose(cube_grid / np.sqrt(3), (1, 2, 3, 0))
cube_rot_grid = np.reshape(cube_rotation.apply(np.reshape(cube_scaled_grid, newshape=(-1, 3))), cube_scaled_grid.shape)

plt.close(fig=5)
plt.figure(num=5).add_subplot(projection='3d')
plt.gca().voxels(cube_rot_grid[..., 0], cube_rot_grid[..., 1], cube_rot_grid[..., 2], 
                 cube[:-1, :-1, :-1], facecolors=colors[1:, 1:, 1:], 
                 shade=False)
plt.gca().scatter([0], [0], [1.0], marker='o')
plt.show()

In [ ]:
# linear equations systems to find intersection points of cube's faces and 
# current ray that rotates around z-axis in parralel to xOy plane
# (6 systems of 3 linear equations for each cube face (with free terms))
equations = np.zeros(shape=(6, 3, 4), dtype=np.float64)

planes = np.zeros(shape=(6, 4), dtype=np.float64)
corners = np.array(cube_rot_grid[0:2, 0:2, 0:2, :] * (edge_len)).reshape((-1, 3)).astype(np.float64)
order = [0, 1, 3, 7, 6, 4, 0, 1] # special path throw cube verticies

# express planes, that contain rotated cube faces, in equations
for p in range(len(order) - 2):
    planes[p][0:3] = np.cross(corners[order[p]] - corners[order[p+1]], corners[order[p+2]] - corners[order[p+1]])
    planes[p][3] = np.sum(planes[p][0:3] * corners[order[p+1]])

equations[:, 0, :] = planes
# HS[I/V/L] cylinder
cylinder_shape = (
    complex(0, int(180 / granularity)),
    complex(0, int((180 / granularity) * 2 * np.pi)),
    complex(0, int(edge_len))
)
cylinder_size = (1.0, 2*np.pi, 1.0)
cylinder_grid = np.mgrid[0:cylinder_size[0]:cylinder_shape[0], 0:cylinder_size[1]:cylinder_shape[1], 0:cylinder_size[2]:cylinder_shape[2]]
cylinder = cylinder_grid[0] < cylinder_size[0]
cylinder_grid = cylinder_grid.transpose((1, 2, 3, 0)).astype(np.float64)
cylinder_colors = np.zeros(shape=cylinder_grid.shape)

for h in range(int(cylinder_shape[2].imag)-1): # decrease range by 1
    center = np.array([0, 0, cylinder_grid[0][0][h][2]])
    equations[:, 1, :] =  np.array([0, 0, 1, cylinder_grid[0][0][h][2]])
    print(f"{center}")
    for t in range(int(cylinder_shape[1].imag)-1): # decrease range by 1
        direction_vector = np.array([np.cos(cylinder_grid[0][t][h][1] - z_axis_rot), -np.sin(cylinder_grid[0][t][h][1] - z_axis_rot), 0])
        equations[:, 2, :] = np.array([np.cos(cylinder_grid[0][t][h][1] - np.pi/2 - z_axis_rot), -np.sin(cylinder_grid[0][t][h][1] - np.pi/2 - z_axis_rot), 0, 0])
        scale_points = np.linalg.solve(equations[:, :, 0:3], np.squeeze(equations[:, :, 3]))
        direction_mask = np.argwhere(np.all(scale_points * direction_vector >= 0, axis=1)) # works well, but np.dot should be used instead of multiplication (or not)
        rate = np.amin(np.linalg.norm(scale_points[direction_mask] - center, axis=2)) / cylinder_size[0]
        for r in range(int(cylinder_shape[0].imag)-1): # decrease range by 1
            cartessian_coordinates = np.array([
                cylinder_grid[r][t][h][0]*np.cos(cylinder_grid[r][t][h][1] + z_axis_rot)*rate,
                cylinder_grid[r][t][h][0]*np.sin(cylinder_grid[r][t][h][1] + z_axis_rot)*rate,
                cylinder_grid[r][t][h][2]
            ])
            index = (cube_rotation.apply(cartessian_coordinates, inverse=True) * (np.sqrt(3) * edge_len)).astype(int)
            cylinder_colors[r][t][h] = colors[index[0]][index[1]][index[2]]


In [ ]:
plt.close(fig=6)
plt.figure(num=6).add_subplot(projection='3d')
# cylinder_grid[:, -1, :, 1] = 0 # does not fix bright plane at angle 0 degrees (red)
plt.gca().voxels(cylinder_grid[..., 0] * np.cos(cylinder_grid[..., 1]), 
                 cylinder_grid[..., 0] * np.sin(cylinder_grid[..., 1]), 
                 cylinder_grid[..., 2], 
                 cylinder[:-1, :-1, :-1], facecolors=cylinder_colors[:-1, :-1, :-1], 
                 shade=False, alpha=0.4)
plt.gca().set_xlabel("x")
plt.gca().set_ylabel("y")
plt.gca().set_zlabel("z")
plt.show()

In [ ]:

cone_shape = (
    complex(0, int(180 / granularity)),
    complex(0, int((180 / granularity) * 2 * np.pi)),
    complex(0, int(edge_len))
)
cone_size = (1.0, 2*np.pi, 1.0)
cone_grid = np.array(cylinder_grid)
cone = cone_grid[..., 0] < cone_grid[..., 2]
cone_surf = np.logical_and(cone_grid[..., 0] > (cone_grid[..., 2] - cone_grid[1, ..., 0]), cone)
cone_surf_sector = np.logical_and(cone_grid[..., 1] < np.pi/2, cone_surf)

# Crack!
cone_grid_factors = np.arange(int(cone_shape[0].imag))
cone_r_decrease_step = cone_grid[1, 0, 0, 0]
cone_r_grid_step = np.empty(int(cone_shape[0].imag), dtype=np.float64)
for h in range(int(cone_shape[2].imag)):
    r = h * (cone_shape[0].imag / cone_shape[2].imag) * (cone_size[0] / cone_size[2])
    r_dis = cone_shape[0].imag - r
    cone_r_grid_step[:int(r_dis+1)] = cone_grid_factors[:int(r_dis+1)] * cone_r_decrease_step
    cone_r_grid_step[int(r_dis+1):] = r_dis * cone_r_decrease_step
    cone_grid[:, :, h, 0] -= np.reshape(cone_r_grid_step, newshape=(-1, 1))

In [ ]:

plt.close(fig=7)
plt.figure(num=7).add_subplot(projection='3d')
plt.gca().voxels(cone_grid[..., 0] * np.cos(cylinder_grid[..., 1]), 
                 cone_grid[..., 0] * np.sin(cylinder_grid[..., 1]), 
                 cone_grid[..., 2], 
                 cylinder[:-1, :-1, :-1], facecolors=cylinder_colors[:-1, :-1, :-1], 
                 shade=False, alpha=0.4)
plt.gca().set_xlabel("x")
plt.gca().set_ylabel("y")
plt.gca().set_zlabel("z")
plt.show()

### *RGB to HS(V/I) conversion:*

\begin{equation}
H = \left[
\begin{split}
& \theta \quad & \textrm{if } B \leq G, \\
& 360^\circ - \theta \quad & \textrm{if } B > G
\end{split}
\right.
\end{equation}

\begin{equation}
\theta = \arccos \left(
    \frac{ \frac{1}{2} \left( \left( R-G \right) + \left( R-B \right) \right) }
        { \left( \left( R-G \right) ^2 + \left( R-B \right) \left( G-B \right) \right) ^ \frac{1}{2}}
\right).
\end{equation}

\begin{equation}
S = 1 - \frac{3}{R+G+B} \min \left( R, G, B \right)
\end{equation}

\begin{equation}
I = \frac{1}{3} \left( R+G+B \right)
\end{equation}

### *RGB to CMY conversion:*

\begin{equation}
\left[ \begin{matrix} c\\m\\y \end{matrix} \right]
= 
\left [\begin{matrix} 1\\1\\1 \end{matrix} \right]
-
\left[ \begin{matrix} r\\g\\b \end{matrix} \right]
\end{equation}

### *RGB to CMYK conversion:*
\begin{equation}
\left[ \begin{matrix} c\\m\\y \end{matrix} \right]
= 
\left [\begin{matrix} 1\\1\\1 \end{matrix} \right]
-
\left[ \begin{matrix} r\\g\\b \end{matrix} \right]
- \min\left({r, g, b}\right)
\end{equation}

\begin{equation}
k = \min\left( r, g, b \right)
\end{equation}

### *CIE LAB definition:*

\begin{equation}
L^* = 116 \cdot h \! \left( \frac{Y}{Y_W} \right) -16, 
\end{equation}

\begin{equation}
a^* = 500 \cdot \left( h \! \left( \frac{X}{X_W} \right) - h \! \left( \frac{Y}{Y_W} \right) \right)
\end{equation}

\begin{equation}
b^* = 200 \cdot \left( h \! \left( \frac{Y}{Y_W} \right) - h \! \left( \frac{Z}{Z_W} \right) \right)
\end{equation}

\begin{equation}
h \! \left( q \right) = \left[
    \begin{split}
        &\sqrt[3]{q} \quad & \textrm{if } q > 0,00856, \\
        &7,787q + 16/116 \quad & \textrm{if } q \leq 0,00856
    \end{split}
\right.
\end{equation}


### *RGB to Y(')CrCb conversion:*

\begin{equation}
Y' = K_R \cdot R' + \left( 1 - K_R - K_B \right) \cdot C' + K_B \cdot B'
\end{equation}

\begin{equation}
P_B = \frac{1}{2} \cdot \frac{B' - Y'}{1 - K_B}
\end{equation}

\begin{equation}
P_B = \frac{1}{2} \cdot \frac{R' - Y'}{1 - K_R}
\end{equation}


In [ ]:
# RGB to CMY
img_n = img.astype(np.float32) / 255
img_cmy = 1 - img_n
# RGB to HSV
img_hsv = np.empty(shape=img.shape)
#__temp = np.sqrt(np.power(img[..., 0] - img[..., 1], 2) + (img[..., 0] - img[..., 2]) * (img[..., 1] - img[..., 2]))
_temp = 0.5 * ((img_n[..., 0] - img_n[..., 1]) + (img_n[..., 0] - img_n[..., 2]))
__temp = np.sqrt(np.power(img_n[..., 0] - img_n[..., 1], 2.0) + (img_n[..., 0] - img_n[..., 2]) * (img_n[..., 1] - img_n[..., 2]))
___temp = np.divide(_temp, __temp, where=__temp != 0)

img_hsv[..., 0] = np.arccos(___temp)
# Normilize angle to range [0, 1]
img_hsv[..., 0] = np.where(img_n[..., 2] <= img_n[..., 1], img_hsv[..., 0], 2*np.pi - img_hsv[..., 0]) / (2 * np.pi)
img_hsv[..., 1] = 1 - 3 / np.sum(img, axis=2) * np.amin(img, axis=2)
img_hsv[..., 2] = np.sum(img_n, axis=2) / 3

# Crack! Good try
plt.close(fig=10)
plt.figure(num=10).add_subplot()
plt.gca().imshow(img_hsv, cmap='hsv')
plt.show()

# Drafts #

In [ ]:
#cube_rotation = scipy.spatial.transform.Rotation.from_euler("XY", (35, -45), degrees=True)
#cube_rotation = scipy.spatial.transform.Rotation.from_euler("xy", (45, -35), degrees=True)

 *Преобразования в разные цветовые схемы: RGB, CMYK, HSV, CIE YCbCr*

\begin{equation}
e^x=\sum_{i=0}^\infty \frac{1}{i!}x^i
\end{equation}

_**Inline formula:**_ $e^{i\pi} + 1 = 0$

In [ ]:
cylinder_shape = (
    complex(0, int(180 / granularity)),
    complex(0, int((180 / granularity) * 2 * np.pi)),
    complex(0, int(edge_len))
)
cylinder_size = (1.0, 2*np.pi, 1.0)
cylinder_grid = np.mgrid[0:cylinder_size[0]:cylinder_shape[0], 0:cylinder_size[1]:cylinder_shape[1], 0:cylinder_size[2]:cylinder_shape[2]]
corners = np.array(hsv_space[0:2, 0:2, 0:2, :] * (edge_len)).reshape((-1, 3)).astype(np.float64)
order = [0, 1, 3, 7, 6, 4, 0, 1]
planes = np.zeros(shape=(6, 4), dtype=np.float64)

for p in range(len(order) - 2):
    #print(f"{p+1}.\t{order[p]}. {corners[order[p]]}\n\t{order[p+1]}. {corners[order[p+1]]}\n\t{order[p+2]}. {corners[order[p+2]]}")
    planes[p][0:3] = np.cross(corners[order[p]] - corners[order[p+1]], corners[order[p+2]] - corners[order[p+1]])
    planes[p][3] = np.sum(planes[p][0:3] * corners[order[p+1]])

equations = np.zeros(shape=(6, 3, 4), dtype=np.float64)
equations[:, 0, :] = planes
#cylinder = ((cylinder_grid[0] * np.cos(cylinder_grid[1])) ** 2 + (cylinder_grid[0] * np.sin(cylinder_grid[1])) ** 2 < 0.25)
cylinder = cylinder_grid[0] < cylinder_size[0]
cylinder_grid = cylinder_grid.transpose((1, 2, 3, 0)).astype(np.float64)
cylinder_colors = np.zeros(shape=cylinder_grid.shape)

border_crossings = []
planed_vectors = []
crossings_dist = []
contained_points = []

__fill_voxels = np.full(cylinder.shape, False, dtype=bool)
for h in range(int(cylinder_shape[2].imag)-1): # decrease range by 1
    center = np.array([0, 0, cylinder_grid[0][0][h][2]])
    print(f"{center}")
    equations[:, 1, :] =  np.array([0, 0, 1, cylinder_grid[0][0][h][2]])
    for t in range(int(cylinder_shape[1].imag)-1): # decrease range by 1
        equations[:, 2, :] = np.array([np.cos(cylinder_grid[0][t][h][1] - np.pi/2 - z_axis_rot), -np.sin(cylinder_grid[0][t][h][1] - np.pi/2 - z_axis_rot), 0, 0])
        scale_points = np.linalg.solve(equations[:, :, 0:3], np.squeeze(equations[:, :, 3]))
        scale_norms = np.linalg.norm(scale_points - center, axis=1)
        direction_vector = np.array([np.cos(cylinder_grid[0][t][h][1] - z_axis_rot), -np.sin(cylinder_grid[0][t][h][1] - z_axis_rot), 0])
        for point in scale_points:
            crossings_dist.append(np.linalg.norm(point - center))
        border_crossings.append(scale_points[np.argmin(scale_norms)])
        border_crossings.append(scale_points[np.argmin(scale_norms)] * np.array([-1, -1, 1]))
        direction_mask = np.argwhere(np.all(scale_points * direction_vector >= 0, axis=1))
        d = np.amin(scale_norms[direction_mask])
        #print(f"{scale_norms}: min={d}; scale_points.shape={scale_points.shape}")
        rate = d / cylinder_size[0]
        for r in range(int(cylinder_shape[0].imag)): # decrease range by 1
            transformed_coordinates = np.array([
                cylinder_grid[r][t][h][0]*np.cos(cylinder_grid[r][t][h][1] + z_axis_rot)*rate,
                cylinder_grid[r][t][h][0]*np.sin(cylinder_grid[r][t][h][1] + z_axis_rot)*rate,
                cylinder_grid[r][t][h][2]
            ])
            contained_points.append(transformed_coordinates)
            index = cube_rotation.apply(transformed_coordinates*np.sqrt(3), inverse=True)
            index *=  edge_len
            index = index.astype(int)

            if np.all(index == np.array([31, 0, 0])):
                print(f"Red: ({r}, {t}, {h})")
            cylinder_colors[r][t][h] = colors[index[0]][index[1]][index[2]]

__boundary_points = np.array(border_crossings)
__contained_points = np.array(contained_points)
__corners = np.reshape(corners, newshape=(2, 2, 2, 3))

plt.close(fig=4)
plt.figure(num=4).add_subplot(projection='3d')
#plt.gca().voxels(cylinder_grid[..., 0] * np.cos(cylinder_grid[..., 1]), cylinder_grid[..., 0] * np.sin(cylinder_grid[..., 1]), cylinder_grid[..., 2], cylinder[:-1, :-1, :-1], facecolors=cylinder_colors[:-1, :-1, :-1])
plt.gca().voxels(__corners[..., 0], __corners[..., 1], __corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
#plt.gca().scatter(__boundary_points[..., 0], __boundary_points[..., 1], __boundary_points[..., 2], marker='o')
plt.gca().scatter(__contained_points[0:-1:3,..., 0], __contained_points[0:-1:3,..., 1], __contained_points[0:-1:3,..., 2], marker='o')
plt.show()

In [ ]:
plt.close(fig=5)
plt.figure(num=5).add_subplot(projection='3d')
plt.gca().voxels(__corners[..., 0], __corners[..., 1], __corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
plt.gca().scatter(__boundary_points[..., 0], __boundary_points[..., 1], __boundary_points[..., 2], marker='o')
plt.show()

In [ ]:
__cp_indicies = np.argwhere(np.logical_and(__contained_points[:, 2] > 0.48, __contained_points[:, 2] < 0.50))
__bp_indicies = np.argwhere(__boundary_points[..., 2] > 0.48)

plt.close(fig=6)
plt.figure(num=6).add_subplot(projection='3d')
plt.gca().scatter(__contained_points[__cp_indicies][0:-1:1, ..., 0], __contained_points[__cp_indicies][0:-1:1, ..., 1], __contained_points[__cp_indicies][0:-1:1, ..., 2], marker='1')
plt.gca().scatter(__boundary_points[__bp_indicies][..., 0], __boundary_points[__bp_indicies][..., 1], __boundary_points[__bp_indicies][..., 2], marker='.')
plt.show()

In [ ]:
__cp_indicies = np.argwhere(np.logical_and(__contained_points[:, 2] > 0.58, __contained_points[:, 2] < 0.60))
__bp_indicies = np.argwhere(__boundary_points[..., 2] > 0.58)

plt.close(fig=7)
plt.figure(num=7).add_subplot(projection='3d')
plt.gca().scatter(__contained_points[__cp_indicies][0:-1:1, ..., 0], __contained_points[__cp_indicies][0:-1:1, ..., 1], __contained_points[__cp_indicies][0:-1:1, ..., 2], marker='1')
plt.gca().scatter(__boundary_points[__bp_indicies][..., 0], __boundary_points[__bp_indicies][..., 1], __boundary_points[__bp_indicies][..., 2], marker='.')
plt.show()

In [ ]:
__r_corners = np.reshape(cube_rotation.apply(np.reshape(__corners, newshape=(-1, 3)), inverse=True), newshape=__corners.shape)
__r_contained_points = cube_rotation.apply(__contained_points, inverse=True)


plt.close(fig=7)
plt.figure(num=7).add_subplot(projection='3d')
plt.gca().voxels(__r_corners[..., 0], __r_corners[..., 1], __r_corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
#plt.gca().scatter(__boundary_points[..., 0], __boundary_points[..., 1], __boundary_points[..., 2], marker='o')
plt.gca().scatter(__r_contained_points[0:-1:3,..., 0], __r_contained_points[0:-1:3,..., 1], __r_contained_points[0:-1:3,..., 2], marker='o')
plt.show()

In [ ]:
__sr_corners = __r_corners * np.sqrt(3) * edge_len
__sr_contained_points = __r_contained_points * np.sqrt(3) * edge_len

plt.close(fig=8)
plt.figure(num=8).add_subplot(projection='3d')
plt.gca().voxels(__sr_corners[..., 0], __sr_corners[..., 1], __sr_corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
#plt.gca().scatter(__boundary_points[..., 0], __boundary_points[..., 1], __boundary_points[..., 2], marker='o')
plt.gca().scatter(__sr_contained_points[0:-1:3,..., 0], __sr_contained_points[0:-1:3,..., 1], __sr_contained_points[0:-1:3,..., 2], marker='o')
plt.show()

In [ ]:
__sr_outband_points_collection = []
for point in __sr_contained_points:
    if(np.any(point < -0)):
        __sr_outband_points_collection.append(point)
__sr_outband_points = np.array(__sr_outband_points_collection)

plt.close(fig=9)
plt.figure(num=9).add_subplot(projection='3d')
plt.gca().voxels(__sr_corners[..., 0], __sr_corners[..., 1], __sr_corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
#plt.gca().scatter(__boundary_points[..., 0], __boundary_points[..., 1], __boundary_points[..., 2], marker='o')
plt.gca().scatter(__sr_outband_points[..., 0], __sr_outband_points[..., 1], __sr_outband_points[..., 2], marker='o')
plt.show()

In [ ]:
__obp_indicies = np.argwhere(__sr_contained_points < 0)
__outband_points = __contained_points[np.argwhere(__sr_contained_points < 0)]
print(__obp_indicies.shape)
print(__outband_points.shape)

plt.close(fig=10)
plt.figure(num=10).add_subplot(projection='3d')
plt.gca().voxels(__corners[..., 0], __corners[..., 1], __corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
plt.gca().scatter(__outband_points[..., 0], __outband_points[..., 1], __outband_points[..., 2], marker='o')
plt.show()

In [ ]:
__line_r_range = np.linspace(-3, 3, num=100)
__line_angle = np.pi / 6
plt.close(fig=11)
plt.figure(num=11)
plt.gca().axis('equal')
plt.gca().grid(True)
plt.gca().plot(__line_r_range * np.cos(__line_angle), __line_r_range * np.sin(__line_angle), label='forward')
plt.gca().plot(__line_r_range * np.cos(__line_angle - np.pi/2), __line_r_range * np.sin(__line_angle - np.pi/2), label='rotated')
plt.gca().plot(__line_r_range * np.cos(__line_angle), -__line_r_range * np.sin(__line_angle), label='inverse')
plt.gcf().legend()
plt.show()

In [ ]:
temp = np.array(border_crossings)
__d_temp = np.array(crossings_dist)
__rc_temp = np.array(planed_vectors)
#print(np.argwhere(__fill_voxels == True))
for i in range(1600, 1600):
    print(f"{__rc_temp[i]}: {__d_temp[i]}")
#temp = np.reshape(temp, newshape=(3, -1))

_border_crossings = np.reshape(np.array(border_crossings), (-1, 3))

surf_base = np.meshgrid(np.linspace(-5, 5, 10), np.linspace(-5, 5, 10))
surf_points = np.zeros(shape=(6, 10, 10, 3))
for i in range(planes.shape[0]):
    surf_points[i, :, :, 0] = surf_base[0]
    surf_points[i, :, :, 1] = surf_base[1]
    surf_points[i, :, :, 2] = (surf_base[0] * planes[i][0] + surf_base[1] * planes[i][1] - planes[i][3]) / (-planes[i][2])

_surf_points = np.reshape(surf_points, (6, -1, 3))

#for j in range(planes.shape[0]):
#    for i in range(planes.shape[0]):
#        print(np.dot(planes[j][0:3], planes[i][0:3]))

plt.close(fig=8)
plt.figure(num=8).add_subplot(projection='3d')\
    .voxels(cylinder_grid[..., 0] * np.cos(cylinder_grid[..., 1]), cylinder_grid[..., 0] * np.sin(cylinder_grid[..., 1]), cylinder_grid[..., 2], cylinder[:-1, :-1, :-1], facecolors=cylinder_colors[:-1, :-1, :-1], shade=False)

#plt.gca().scatter(temp[0:100, 0], temp[0:100, 1], temp[0:100, 2], marker='^')
surf_cmaps = ["Greens", "Reds", "Blues", "ocean", "Oranges", "Greys"]

for i in []:
    plt.gca().plot_surface(surf_points[i][:, :, 0], surf_points[i][:, :, 1], surf_points[i][:, :, 2], cmap=surf_cmaps[i])
#for p in range(surf_points.shape[0]):
#    plt.gca().plot_surface(surf_points[p][:, :, 0], surf_points[p][:, :, 1], surf_points[p][:, :, 2], cmap=surf_cmaps[p])
plt.show()

In [ ]:
print(cylinder_grid[:, -1, :, 0] * np.cos(cylinder_grid[:, -1, :, 1]) - cylinder_grid[:, 0, :, 0] * np.cos(cylinder_grid[:, 0, :, 1]))